Source: [Harry Potter Dataset](https://www.kaggle.com/datasets/gulsahdemiryurek/harry-potter-dataset)

### Import libraries

In [645]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re

In [646]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load data

In [647]:
data_houses = pd.read_csv('Characters.csv', sep=";")

In [648]:
data_houses.shape

(140, 15)

In [649]:
data_houses.head(3)

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,Parseltongue| Defence Against the Dark Arts | Seeker,31 July 1980,NaN
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,Almost everything,"19 September, 1979",NaN


# Clean data

From existing columns, I extract clean data:
- split name into first name and last name
- convert gender to boolean values (0 or 1)
- select people who are 'student' and 'teacher' (Headmaster or Professor), 
- split full information about wand into length, wood, core source and core element
- extract only animal species from patronus
- select people who are considered as: human, giant, werewolf, goblin, ghost
- select people with specific blood status: pure, half and muggle-born
- extract only colors (without tone) from hair and eye colours
- select people who are loyal to specific groups: Dumbledore's Army, Order of the Phoenix and Lord Voldemort/Death eaters
- select people with special skills: is highly skilled, is a student, is a prefect, have something in common with quidditch, have something in common with unforgivable curses
- convert Hogwart's houses names to numeric values

In [650]:
new_cols = [col.strip().replace(' ','_').lower() for col in data_houses.columns]
data_houses.rename(columns={key:val for (key,val) in zip(data_houses.columns,new_cols)},inplace=True)

data_houses = data_houses[data_houses.columns[1:]]

In [651]:
print('Data columns types:')
display(data_houses.dtypes)

Data columns types:


name            object
gender          object
job             object
house           object
wand            object
patronus        object
species         object
blood_status    object
hair_colour     object
eye_colour      object
loyalty         object
skills          object
birth           object
death           object
dtype: object

In [652]:
print('Object variables:')
display(data_houses.describe(include = [object]))

Object variables:


,name,gender,job,house,wand,patronus,species,blood_status,hair_colour,eye_colour,loyalty,skills,birth,death
count,140,139,121,101,132,130,140,123,123,86,89,113,127,42
unique,140,2,65,6,29,20,10,15,36,25,19,94,112,25
top,Harry James Potter,Male,Student,Gryffindor,Unknown,Unknown,Human,Pure-blood or half-blood,Black,Brown,Order of the Phoenix,Chaser,Pre 976,"2 May, 1998"
freq,1,90,52,38,104,75,105,38,25,16,16,7,4,9


In [653]:
# create a list of columns names which will be use to do the following search
new_columns = []

In [654]:
# split full names into new columns: 'first_name' and 'last_name'
i = data_houses.columns.get_loc('name')
data_houses.insert(i+1, 'first_name', data_houses['name'].str.split().str[0].str.strip('()'))
data_houses.insert(i+2, 'last_name', data_houses['name'].str.replace('Sr.','').str.replace('Jr.','').str.split().str[-1].str.strip('()'))
new_columns.append('first_name')
new_columns.append('last_name')

data_houses[['name','first_name','last_name']]

C:\Users\Martyna\AppData\Local\Temp\ipykernel_9044\727689816.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data_houses.insert(i+2, 'last_name', data_houses['name'].str.replace('Sr.','').str.replace('Jr.','').str.split().str[-1].str.strip('()'))


,name,first_name,last_name
0,Harry James Potter,Harry,Potter
1,Ronald Bilius Weasley,Ronald,Weasley
2,Hermione Jean Granger,Hermione,Granger
3,Albus Percival Wulfric Brian Dumbledore,Albus,Dumbledore
4,Rubeus Hagrid,Rubeus,Hagrid
5,Neville Longbottom,Neville,Longbottom
6,Fred Weasley,Fred,Weasley
7,George Weasley,George,Weasley
8,Ginevra (Ginny) Molly Weasley,Ginevra,Weasley
9,Dean Thomas,Dean,Thomas


In [655]:
# map gender column:
#   - 0: Male
#   - 1: Female
mapper = {'Male': 0, 'Female': 1}
i = data_houses.columns.get_loc('gender')
data_houses.insert(i+1, 'sex', data_houses['gender'].map(mapper))
new_columns.append('sex')

In [656]:
def col_contains_words(df,col,key_words):
    # df - data frame
    # col - column name (string)
    # key_words - wanted words (list of strings)   
    return pd.Series(any(w.lower() in str(line).lower() for w in key_words) if not str(line)=='nan' else np.nan for line in df[col])*1

In [657]:
# split 'job' column into two new columns: 'student' and 'teacher' (Headmaster or Professor)
student = col_contains_words(data_houses,'job',['student'])
teacher = col_contains_words(data_houses,'job',['headmaster','professor'])

i = data_houses.columns.get_loc('job')
data_houses.insert(i+1, 'student', student)
data_houses.insert(i+2, 'teacher', teacher)

new_columns.append('student')
new_columns.append('teacher')

data_houses[['name','job','student','teacher']]

,name,job,student,teacher
0,Harry James Potter,Student,1,0
1,Ronald Bilius Weasley,Student,1,0
2,Hermione Jean Granger,Student,1,0
3,Albus Percival Wulfric Brian Dumbledore,Headmaster,0,1
4,Rubeus Hagrid,Keeper of Keys and Grounds | Professor of Care of Magical Creatures,0,1
5,Neville Longbottom,Student,1,0
6,Fred Weasley,Student,1,0
7,George Weasley,Student,1,0
8,Ginevra (Ginny) Molly Weasley,Student,1,0
9,Dean Thomas,Student,1,0


In [658]:
def col_extract_words(df,col,words):
    # df - data frame
    # col - column name (string)
    # words - words to extract (list of strings)
    chars = '!@#$%^&*()_+=-,./<>?;:"\|][{}'
    new_col = []
    for line in df[col]:           
        line_words = [word.lower().strip(chars) for word in str(line).split()]
        inter = list(set(line_words)&set(words))
        if len(inter)<1:
            new_col.append(np.nan)
        else:
            new_col.append(inter[0])
    return pd.Series(new_col)

In [659]:
# split wand characteristic into new columns: 'length', 'wood' and 'core'
wand_length = []
for wand in data_houses['wand']:
    length = [s for s in str(wand) if s.isdigit()]        
    if len(length)==1:
        wand_length.append(int(length[0]))
    elif len(length)==2:
        wand_length.append(int(length[0]+length[1]))
    else:
        wand_length.append(np.nan)        
wand_length = pd.Series(wand_length)

woods = ['acacia','alder','apple','ash','aspen','beech','birch','black','blackthorn','cedar','cherry','chestnut','cypress',
         'dogwood','ebony','elder','elm','english','fir','hawthorn','hazel','holly','hornbeam','larch','laurel','lime',
         'mahogany','maple','oak','olive','pear','pine','poplar','red','redwood','rosewood','rowan','silver','snakewood',
         'spruce','sycamore','vine','walnut','willow','yew']
cores_source = ['basilisk','cat','coral','dittany','dragon','jackalope','kelpie','kneazle','monster','phoenix','river',
                'rougarou','serpent','snallygaster','thestral','thunderbird','troll','unicorn','veela','wampus']
cores_part = ['antler','bone','feather','hair','heartstring','horn','mane','shell','spine','stalk','whisker','white']

wand_wood = col_extract_words(data_houses,'wand',woods)
wand_core_src = col_extract_words(data_houses,'wand',cores_source)
wand_core_part = col_extract_words(data_houses,'wand',cores_part)

In [660]:
pd.DataFrame({'name':data_houses.name,'wand':data_houses.wand,'wand_length':wand_length,'wand_wood':wand_wood,
              'wand_core_src':wand_core_src,'wand_core_part':wand_core_part})

,name,wand,wand_length,wand_wood,wand_core_src,wand_core_part
0,Harry James Potter,"11"" Holly phoenix feather",11.0,holly,phoenix,feather
1,Ronald Bilius Weasley,"12"" Ash unicorn tail hair",12.0,ash,unicorn,hair
2,Hermione Jean Granger,"10¾"" vine wood dragon heartstring",10.0,vine,dragon,heartstring
3,Albus Percival Wulfric Brian Dumbledore,"15"" Elder Thestral tail hair core",15.0,elder,thestral,hair
4,Rubeus Hagrid,"16"" Oak unknown core",16.0,oak,NaN,NaN
5,Neville Longbottom,"13"" Cherry unicorn hair",13.0,cherry,unicorn,hair
6,Fred Weasley,Unknown,NaN,NaN,NaN,NaN
7,George Weasley,Unknown,NaN,NaN,NaN,NaN
8,Ginevra (Ginny) Molly Weasley,Unknown,NaN,NaN,NaN,NaN
9,Dean Thomas,Unknown,NaN,NaN,NaN,NaN


In [661]:
# convert column with strings values into numeric values
mapper = {}
def str_to_int(col, name, dictonary=None):
    # col - column (pd.Series) with string values to convert
    # name - key for the mapper dictionary (to append col as the value)
    # optional dictionary
    unique_val = col.unique().tolist()
    if dictonary == None:
        dictonary = {key:val for key,val in zip(unique_val,range(len(unique_val)))}
    elif len(unique_val)>len(dictonary.keys()):
        for key in unique_val:
            if not key in dictonary.keys():
                dictonary[key] = -1
    dictonary[np.nan] = np.nan
    print(name + ' mapper:')
    print(dictonary)
    mapper[name]=dictonary
    return pd.Series(col).map(dictonary, na_action='ignore')

In [662]:
# convert wand_wood, wand_core_src and wand_core_part strings into numeric values
wand_wood = str_to_int(wand_wood, 'wand_wood')
wand_core_src = str_to_int(wand_core_src, 'wand_core_src')
wand_core_part = str_to_int(wand_core_part, 'wand_core_part')

i = data_houses.columns.get_loc('wand')
data_houses.insert(i+1, 'wand_length', wand_length)
data_houses.insert(i+2, 'wand_wood', wand_wood)
data_houses.insert(i+3, 'wand_core_src', wand_core_src)
data_houses.insert(i+4, 'wand_core_part', wand_core_part)
new_columns.append('wand_length')
new_columns.append('wand_wood')
new_columns.append('wand_core_src')
new_columns.append('wand_core_part')

wand_wood mapper:
{'holly': 0, 'ash': 1, 'vine': 2, 'elder': 3, 'oak': 4, 'cherry': 5, nan: nan, 'willow': 7, 'mahogany': 8, 'cypress': 9, 'chestnut': 10, 'fir': 11, 'alder': 12, 'hazel': 13, 'hornbeam': 14, 'hawthorn': 15, 'walnut': 16, 'birch': 17, 'cedar': 18, 'elm': 19, 'yew': 20, 'snakewood': 21, 'rosewood': 22}
wand_core_src mapper:
{'phoenix': 0, 'unicorn': 1, 'dragon': 2, 'thestral': 3, nan: nan, 'basilisk': 5, 'veela': 6}
wand_core_part mapper:
{'feather': 0, 'hair': 1, 'heartstring': 2, nan: nan, 'horn': 4}


In [663]:
# extract only species from patronus column and convert strings into numeric values
patronus = data_houses['patronus'].str.lower().str.split().str[-1]
patronus = str_to_int(patronus, 'patronus')
i = data_houses.columns.get_loc('patronus')
data_houses.insert(i+1, 'patronus_n', patronus)
new_columns.append('patronus_n')

data_houses[['name','patronus','patronus_n']]

patronus mapper:
{'stag': 0, 'terrier': 1, 'otter': 2, 'phoenix': 3, 'none': 4, 'non-corporeal': 5, 'unknown': 6, 'horse': 7, 'fox': 8, 'doe': 9, 'wolf': 10, 'cat': 11, 'weasel': 12, 'swan': 13, 'hare': 14, 'squirrel': 15, nan: nan, 'boar': 17, 'lynx': 18, 'goat': 19}


,name,patronus,patronus_n
0,Harry James Potter,Stag,0.0
1,Ronald Bilius Weasley,Jack Russell terrier,1.0
2,Hermione Jean Granger,Otter,2.0
3,Albus Percival Wulfric Brian Dumbledore,Phoenix,3.0
4,Rubeus Hagrid,None,4.0
5,Neville Longbottom,Non-corporeal,5.0
6,Fred Weasley,Unknown,6.0
7,George Weasley,Unknown,6.0
8,Ginevra (Ginny) Molly Weasley,Horse,7.0
9,Dean Thomas,Unknown,6.0


In [664]:
# select people who are considered as: 'human', 'giant', 'werewolf', 'goblin', 'ghost' (True or False)
human = col_contains_words(data_houses,'species',['human'])
giant = col_contains_words(data_houses,'species',['giant'])
werewolf = col_contains_words(data_houses,'species',['werewolf'])
goblin = col_contains_words(data_houses,'species',['goblin'])
ghost = col_contains_words(data_houses,'species',['ghost'])

i = data_houses.columns.get_loc('species')
data_houses.insert(i+1, 'human', human)
data_houses.insert(i+2, 'giant', giant)
data_houses.insert(i+3, 'werewolf', werewolf)
data_houses.insert(i+4, 'goblin', goblin)
data_houses.insert(i+5, 'ghost', ghost)
new_columns.append('human')
new_columns.append('giant')
new_columns.append('werewolf')
new_columns.append('goblin')
new_columns.append('ghost')

data_houses[['name','first_name','species','human','giant','werewolf','goblin','ghost']]

,name,first_name,species,human,giant,werewolf,goblin,ghost
0,Harry James Potter,Harry,Human,1,0,0,0,0
1,Ronald Bilius Weasley,Ronald,Human,1,0,0,0,0
2,Hermione Jean Granger,Hermione,Human,1,0,0,0,0
3,Albus Percival Wulfric Brian Dumbledore,Albus,Human,1,0,0,0,0
4,Rubeus Hagrid,Rubeus,Half-Human/Half-Giant,1,1,0,0,0
5,Neville Longbottom,Neville,Human,1,0,0,0,0
6,Fred Weasley,Fred,Human,1,0,0,0,0
7,George Weasley,George,Human,1,0,0,0,0
8,Ginevra (Ginny) Molly Weasley,Ginevra,Human,1,0,0,0,0
9,Dean Thomas,Dean,Human,1,0,0,0,0


In [665]:
# select people with specific blood status into new columns: 'pure_blood', 'half_blood' and 'muggle_born' 
# (True or False, if the person is a member of this blood group, or is considered as one)
pure_blood = col_contains_words(data_houses,'blood_status',['pure'])
half_blood = col_contains_words(data_houses,'blood_status',['half'])
muggle_born = col_contains_words(data_houses,'blood_status',['muggle'])

i = data_houses.columns.get_loc('blood_status')
data_houses.insert(i+1, 'pure_blood', pure_blood)
data_houses.insert(i+2, 'half_blood', half_blood)
data_houses.insert(i+3, 'muggle_born', muggle_born)

new_columns.append('pure_blood')
new_columns.append('half_blood')
new_columns.append('muggle_born')

data_houses[['name','first_name','blood_status','pure_blood','half_blood','muggle_born']]

,name,first_name,blood_status,pure_blood,half_blood,muggle_born
0,Harry James Potter,Harry,Half-blood,0,1,0
1,Ronald Bilius Weasley,Ronald,Pure-blood,1,0,0
2,Hermione Jean Granger,Hermione,Muggle-born,0,0,1
3,Albus Percival Wulfric Brian Dumbledore,Albus,Half-blood,0,1,0
4,Rubeus Hagrid,Rubeus,Part-Human (Half-giant),0,1,0
5,Neville Longbottom,Neville,Pure-blood,1,0,0
6,Fred Weasley,Fred,Pure-blood,1,0,0
7,George Weasley,George,Pure-blood,1,0,0
8,Ginevra (Ginny) Molly Weasley,Ginevra,Pure-blood,1,0,0
9,Dean Thomas,Dean,Muggle-born,0,0,1


In [666]:
# extract only colours (without tone) from hair colours
hair_black = col_contains_words(data_houses,'hair_colour',['black','dark'])
hair_red = col_contains_words(data_houses,'hair_colour',['red'])
hair_brown = col_contains_words(data_houses,'hair_colour',['brown','auburn'])
hair_blond = col_contains_words(data_houses,'hair_colour',['blond','blonde','sandy'])
hair_grey = col_contains_words(data_houses,'hair_colour',['grey','silver','mousy','white'])
hair_bald = col_contains_words(data_houses,'hair_colour',['bald'])

i = data_houses.columns.get_loc('hair_colour')
data_houses.insert(i+1, 'hair_black', hair_black)
data_houses.insert(i+2, 'hair_red', hair_red)
data_houses.insert(i+3, 'hair_brown', hair_brown)
data_houses.insert(i+4, 'hair_blond', hair_blond)
data_houses.insert(i+5, 'hair_grey', hair_grey)
data_houses.insert(i+6, 'hair_bald', hair_bald)

new_columns.append('hair_black')
new_columns.append('hair_red')
new_columns.append('hair_brown')
new_columns.append('hair_blond')
new_columns.append('hair_grey')
new_columns.append('hair_bald')

data_houses[['name','first_name','hair_colour','hair_black','hair_red','hair_brown','hair_blond','hair_grey','hair_bald']]

,name,first_name,hair_colour,hair_black,hair_red,hair_brown,hair_blond,hair_grey,hair_bald
0,Harry James Potter,Harry,Black,1,0,0,0,0,0
1,Ronald Bilius Weasley,Ronald,Red,0,1,0,0,0,0
2,Hermione Jean Granger,Hermione,Brown,0,0,1,0,0,0
3,Albus Percival Wulfric Brian Dumbledore,Albus,Silver| formerly auburn,0,0,1,0,1,0
4,Rubeus Hagrid,Rubeus,Black,1,0,0,0,0,0
5,Neville Longbottom,Neville,Blond,0,0,0,1,0,0
6,Fred Weasley,Fred,Red,0,1,0,0,0,0
7,George Weasley,George,Red,0,1,0,0,0,0
8,Ginevra (Ginny) Molly Weasley,Ginevra,Red,0,1,0,0,0,0
9,Dean Thomas,Dean,Black,1,0,0,0,0,0


In [667]:
data_houses[data_houses.hair_bald==True][['name','first_name','hair_colour','hair_black','hair_red','hair_brown','hair_blond','hair_grey','hair_bald']]

,name,first_name,hair_colour,hair_black,hair_red,hair_brown,hair_blond,hair_grey,hair_bald
15,Peter Pettigrew,Peter,Colourless and balding,0,0,0,0,0,1
51,Horace Eugene Flaccus Slughorn,Horace,Bald,0,0,0,0,0,1
57,Tom Marvolo Riddle,Tom,Bald,0,0,0,0,0,1
71,Cuthbert Binns,Cuthbert,White (balding),0,0,0,0,1,1
94,Kingsley Shacklebolt,Kingsley,Bald,0,0,0,0,0,1


In [668]:
# extract only colours (without tone) from eye colours
eye_green = col_contains_words(data_houses,'eye_colour',['green','gooseberry'])
eye_blue = col_contains_words(data_houses,'eye_colour',['blue'])
eye_brown = col_contains_words(data_houses,'eye_colour',['brown'])
eye_hazel = col_contains_words(data_houses,'eye_colour',['hazel', 'yellow'])
eye_black = col_contains_words(data_houses,'eye_colour',['black'])
eye_grey = col_contains_words(data_houses,'eye_colour',['grey','silver'])
eye_scarlet = col_contains_words(data_houses,'eye_colour',['scarlet'])

i = data_houses.columns.get_loc('eye_colour')
data_houses.insert(i+1, 'eye_green', eye_green)
data_houses.insert(i+2, 'eye_blue', eye_blue)
data_houses.insert(i+3, 'eye_brown', eye_brown)
data_houses.insert(i+4, 'eye_hazel', eye_hazel)
data_houses.insert(i+5, 'eye_black', eye_black)
data_houses.insert(i+6, 'eye_grey', eye_grey)
data_houses.insert(i+7, 'eye_scarlet', eye_scarlet)

new_columns.append('eye_green')
new_columns.append('eye_blue')
new_columns.append('eye_brown')
new_columns.append('eye_hazel')
new_columns.append('eye_black')
new_columns.append('eye_grey')
new_columns.append('eye_scarlet')

data_houses[['name','eye_colour','eye_green','eye_blue','eye_brown','eye_hazel','eye_black','eye_grey','eye_scarlet']]

,name,eye_colour,eye_green,eye_blue,eye_brown,eye_hazel,eye_black,eye_grey,eye_scarlet
0,Harry James Potter,Bright green,1,0,0,0,0,0,0
1,Ronald Bilius Weasley,Blue,0,1,0,0,0,0,0
2,Hermione Jean Granger,Brown,0,0,1,0,0,0,0
3,Albus Percival Wulfric Brian Dumbledore,Blue,0,1,0,0,0,0,0
4,Rubeus Hagrid,Black,0,0,0,0,1,0,0
5,Neville Longbottom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fred Weasley,Brown,0,0,1,0,0,0,0
7,George Weasley,Brown,0,0,1,0,0,0,0
8,Ginevra (Ginny) Molly Weasley,Bright brown,0,0,1,0,0,0,0
9,Dean Thomas,Brown,0,0,1,0,0,0,0


In [669]:
# point out info about loyalty into new columns: 'order_of_the_phoenix', 'dumbledores_army' and 'death_eater', 'hogwarts'
# (True or False, if the person is loyal to the specific group)

order_of_the_phoenix = col_contains_words(data_houses,'loyalty',['order','phoenix'])
dumbledores_army = col_contains_words(data_houses,'loyalty',["dumbledore's army", 'army'])
death_eater = col_contains_words(data_houses,'loyalty',['death','eater','voldemort'])
hogwarts = col_contains_words(data_houses,'loyalty',['hogwart'])

i = data_houses.columns.get_loc('loyalty')
data_houses.insert(i+1, 'order_of_the_phoenix', order_of_the_phoenix)
data_houses.insert(i+2, 'dumbledores_army', dumbledores_army)
data_houses.insert(i+3, 'death_eater', death_eater)
data_houses.insert(i+4, 'hogwarts', hogwarts)

new_columns.append('order_of_the_phoenix')
new_columns.append('dumbledores_army')
new_columns.append('death_eater')
new_columns.append('hogwarts')

data_houses[['name','loyalty','order_of_the_phoenix','dumbledores_army','death_eater','hogwarts']]

,name,loyalty,order_of_the_phoenix,dumbledores_army,death_eater,hogwarts
0,Harry James Potter,Albus Dumbledore | Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
1,Ronald Bilius Weasley,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
2,Hermione Jean Granger,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
3,Albus Percival Wulfric Brian Dumbledore,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
4,Rubeus Hagrid,Albus Dumbledore | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,0,0,1
5,Neville Longbottom,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
6,Fred Weasley,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
7,George Weasley,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
8,Ginevra (Ginny) Molly Weasley,Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry,1,1,0,1
9,Dean Thomas,Dumbledore's Army | Hogwarts School of Witchcraft and Wizardry,0,1,0,1


In [670]:
# point out info about characteristic skills into new columns: 
#   - 'skilled' (if this person is highly skilled or not - True/False)
#   - 'prefect' (if this person is/was a prefect or not - True/False)
#   - 'auror' (if this person is/was a auror or not - True/False)
#   - 'quidditch' (if this person is correlated with quidditch or not - True/False)
#   - 'curse' (if this person s correlated with unforgivable curses or not - True/False)

skilled_words = ['highly','skill','talented', 'exceptionally','outstanding','powerful','most','above average']
skilled = col_contains_words(data_houses,'skills',skilled_words)

prefect = col_contains_words(data_houses,'skills',['prefect'])
auror = col_contains_words(data_houses,'skills',['auror'])

quidditch_words = ['quidditch', 'chaser', 'beater', 'keeper', 'seeker', 'quaffle', 'bludger', 'snitch', 'captain']
quidditch = col_contains_words(data_houses,'skills',quidditch_words)

unforgivable_curse = ['avada','kedavra','cruciatus','imperius']
curse = col_contains_words(data_houses,'skills',unforgivable_curse)

i = data_houses.columns.get_loc('skills')
data_houses.insert(i+1, 'skilled', skilled)
data_houses.insert(i+2, 'prefect', prefect)
data_houses.insert(i+3, 'auror', auror)
data_houses.insert(i+4, 'quidditch', quidditch)
data_houses.insert(i+5, 'curse', curse)

new_columns.append('skilled')
new_columns.append('prefect')
new_columns.append('auror')
new_columns.append('quidditch')
new_columns.append('curse')

In [671]:
data_houses[data_houses['curse']==True][['name','skills','skilled','prefect','auror','quidditch','curse']]

,name,skills,skilled,prefect,auror,quidditch,curse
76,Amycus Carrow,Cruciatus Curse,0,0,0,0,1
92,Corban Yaxley,"Imperius curse, undercover work",0,0,0,0,1
100,Barty Crouch Sr.,"Multilingual, able to fight the Imperius curse",0,0,0,0,1


In [672]:
# convert Hogwart Houses names to numeric values
house_dict = {'Ravenclaw': 1, 'Hufflepuff': 2, 'Slytherin': 3, 'Gryffindor': 4}
house = str_to_int(col=data_houses.house, name='house', dictonary=house_dict)
i = len(data_houses.columns)
data_houses.insert(i, 'Y_house', house)
new_columns.append('Y_house')

house mapper:
{'Ravenclaw': 1, 'Hufflepuff': 2, 'Slytherin': 3, 'Gryffindor': 4, nan: nan, 'Beauxbatons Academy of Magic': -1, 'Durmstrang Institute': -1}


In [673]:
clean_data = data_houses[new_columns]
clean_data

,first_name,last_name,sex,student,teacher,wand_length,wand_wood,wand_core_src,wand_core_part,patronus_n,human,giant,werewolf,goblin,ghost,pure_blood,half_blood,muggle_born,hair_black,hair_red,hair_brown,hair_blond,hair_grey,hair_bald,eye_green,eye_blue,eye_brown,eye_hazel,eye_black,eye_grey,eye_scarlet,order_of_the_phoenix,dumbledores_army,death_eater,hogwarts,skilled,prefect,auror,quidditch,curse,Y_house
0,Harry,Potter,0.0,1,0,11.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,4.0
1,Ronald,Weasley,0.0,1,0,12.0,1.0,1.0,1.0,1.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,1,0,4.0
2,Hermione,Granger,1.0,1,0,10.0,2.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,4.0
3,Albus,Dumbledore,0.0,0,1,15.0,3.0,3.0,1.0,3.0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,4.0
4,Rubeus,Hagrid,0.0,0,1,16.0,4.0,NaN,NaN,4.0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,4.0
5,Neville,Longbottom,0.0,1,0,13.0,5.0,1.0,1.0,5.0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,1,0,0,0,0,0,4.0
6,Fred,Weasley,0.0,1,0,NaN,NaN,NaN,NaN,6.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,4.0
7,George,Weasley,0.0,1,0,NaN,NaN,NaN,NaN,6.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,4.0
8,Ginevra,Weasley,1.0,1,0,NaN,NaN,NaN,NaN,7.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,4.0
9,Dean,Thomas,0.0,1,0,NaN,NaN,NaN,NaN,6.0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,4.0


### Export clean data and mappers to csv files

In [674]:
clean_data.to_csv('hogwart_houses.csv')

In [675]:
for mapper_dict in mapper:
    name = mapper_dict+'_dictionary'
    df = pd.Series(mapper[mapper_dict],name='value').to_frame().reset_index().rename(columns={'index':mapper_dict})
    filename=name+'.csv'
    df.to_csv(filename)